In [ ]:
import os
import gc
import math
import time
import random
import scipy as sp
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from dataclasses import dataclass
from typing import Optional
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, AutoConfig, AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(seed=2019)
#print (torch.cuda.manual_seed(2019))

In [ ]:
@dataclass(frozen=True)
class CFG:
    num_workers: Optional[int] = 4
    config_path: Optional[str] = '../input/robertalarge'
    model_path: Optional[str] = '../input/phrase-matching-roberta-training-pytorch-wandb'
    model_name: Optional[str] = 'roberta-large'
    batch_size: Optional[int] = 32
    max_len: Optional[int] = 128
    seed: Optional[int] = 2019
    num_targets: Optional[int] = 1
    n_folds: Optional[int] = 5
    tokenizer = AutoTokenizer.from_pretrained('../input/robertalarge')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('../input/robertalarge')
config_path: Optional[str] = '../input/robertalarge'
print((tokenizer))
print(type(tokenizer))

In [ ]:
PATH = '../input/us-patent-phrase-to-phrase-matching'
test = pd.read_csv(os.path.join(PATH, 'test.csv'))
sub = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))
print(test)
#sub.head()

In [ ]:
context_mapping = {
        "A": "Human Necessities",
        "B": "Operations and Transport",
        "C": "Chemistry and Metallurgy",
        "D": "Textiles",
        "E": "Fixed Constructions",
        "F": "Mechanical Engineering",
        "G": "Physics",
        "H": "Electricity",
        "Y": "Emerging Cross-Sectional Technologies",
}
    
test.context = test.context.apply(lambda x: context_mapping[x[0]])
print(test)

In [ ]:
class PhraseDataset:
    def __init__(self, anchor, target, context, tokenizer, max_len):
        self.anchor = anchor
        self.target = target
        self.context = context
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.anchor)

    def __getitem__(self, item):
        anchor = self.anchor[item]
        context = self.context[item]
        target = self.target[item]

        encoded_text = CFG.tokenizer.encode_plus(
            context + " " + anchor,
            target,
            padding="max_length",
            max_length=self.max_len,
            truncation=True,
        )
        input_ids = encoded_text["input_ids"]
        attention_mask = encoded_text["attention_mask"]

        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
        }

In [ ]:
def inference_fn(model, test_loader):  
    model.eval()
    predictions = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    for data in tk0:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        with torch.no_grad():
            output = model(ids, mask)
        predictions.append(output.sigmoid().detach().cpu().numpy())
        
    return np.concatenate(predictions)

In [ ]:
class PatentModel(torch.nn.Module):
    def __init__(self):
        super(PatentModel, self).__init__()
        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(CFG.config_path)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
            }
        )
        
        self.transformer = AutoModel.from_pretrained(CFG.config_path, config=config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        self.output = nn.Linear(config.hidden_size, CFG.num_targets)
        
    def forward(self, ids, mask):
        transformer_out = self.transformer(input_ids=ids, attention_mask=mask)
        last_hidden_states = transformer_out[0]
        last_hidden_states = self.dropout(torch.mean(last_hidden_states, 1))
        logits1 = self.output(self.dropout1(last_hidden_states))
        logits2 = self.output(self.dropout2(last_hidden_states))
        logits3 = self.output(self.dropout3(last_hidden_states))
        logits4 = self.output(self.dropout4(last_hidden_states))
        logits5 = self.output(self.dropout5(last_hidden_states))
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        
        return logits

In [ ]:
def run_fold(test, fold, seed=42):    
    
    seed_everything(seed)
    
    test_dataset = PhraseDataset(
        test.anchor.values,
        test.target.values,
        test.context.values,
        CFG.tokenizer, 
        CFG.max_len
    ) 
    
    test_loader = DataLoader(test_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    model = PatentModel()
    
    model.load_state_dict(
        torch.load(f'{CFG.model_path}/{CFG.model_name.replace("-","_")}_patent_model_{fold}.pth',
        map_location=torch.device('cuda')
        )
    )
    
    model.to(device)

    preds = inference_fn(model, test_loader)
    
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return preds

In [ ]:
def inference_model(test, seed):
    
    predictions = []
    
    for f in range(CFG.n_folds):    
        preds = run_fold(test, f, seed) 
        predictions.append(preds)
        
    test_preds = np.column_stack(predictions)
        
    return test_preds

In [ ]:
if __name__ == '__main__':
    final_predictions =  inference_model(test, CFG.seed) 

In [ ]:
sub['score'] = np.mean(final_predictions, axis=1)
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head()